# Library import

In [ ]:
git clone https://github.com/ultralytics/yolov5  # clone
cd yolov5
pip install -r requirements.txt  # install

In [ ]:
import os

import yaml

import torch
from torch.utils.data import DataLoader
from torch.optim import Adam
from tqdm import tqdm
import torch.nn as nn


## Path

In [ ]:
# path per la generazione del file .yalm da utilizzare per il finetuning 
root_path =
train_dir = 
val_dir =
test_dir = 

# path per la gestione del modello e dell'addestramento
model_path="yolov5s.pt"

# Dataloader

In [ ]:
# il dataset etichettato deve essere elaborato con Roboflow per organizzare i dati ed esportarli in formatoYOLOv5

## Scrivere un dataset.yaml che definisce i percorsi treno/val e i nomi delle classi

In [ ]:
class DatasetConfigCreator: #rivedere -> è provvisorio
    """
    Classe per creare un file di configurazione YAML per un dataset YOLO.
    """
    def __init__(self, root_path, train_dir, val_dir, test_dir=None, class_names=None):
        """
        Inizializza il DatasetConfigCreator.

        Args:
            root_path (str): Percorso radice del dataset.
            train_dir (str): Directory delle immagini di training relative al percorso radice.
            val_dir (str): Directory delle immagini di validazione relative al percorso radice.
            test_dir (str, optional): Directory delle immagini di test relative al percorso radice. Default è None.
            class_names (list, optional): Lista di nomi delle classi. Default è None.
        """
        self.root_path = root_path
        self.train_dir = train_dir
        self.val_dir = val_dir
        self.test_dir = test_dir
        self.class_names = class_names if class_names else []

    def create_config(self, output_path="dataset.yaml"):
        """
        Crea il file YAML di configurazione.

        Args:
            output_path (str): Percorso del file YAML da generare. Default è "dataset.yaml".
        """
        config = {
            "path": self.root_path,
            "train": self.train_dir,
            "val": self.val_dir,
        }

        # Aggiungi la sezione test solo se specificata
        if self.test_dir:
            config["test"] = self.test_dir

        # Aggiungi le classi
        config["names"] = {i: name for i, name in enumerate(self.class_names)}

        # Verifica se il file esiste
        if os.path.exists(output_path):
            print(f"Il file '{output_path}' esiste già. Nessuna modifica è stata apportata.")
        else:
            # Scrive il file YAML
            with open(output_path, "w") as file:
                yaml.dump(config, file, default_flow_style=False)
            print(f"File di configurazione creato: {output_path}")


# Network

In [ ]:
class YoloModel(nn.Module):
    """
    Classe YOLOv5 per definire il modello e la funzione di forward.
    """
    def __init__(self, model_path="yolov5s.pt"):
        """
        Inizializza il modello YOLOv5.

        Args:
            model_path (str): Percorso ai pesi pre-addestrati YOLOv5.
        """
        super(YoloModel, self).__init__()
        self.model_path = model_path
        self.model = torch.hub.load('ultralytics/yolov5', 'custom', path=model_path)

    def forward(self, images):
        """
        Esegue la predizione sul batch di immagini.

        Args:
            images (torch.Tensor): Batch di immagini di input.

        Returns:
            torch.Tensor: Risultati delle predizioni.
        """
        return self.model(images)


## per addestrare il modello
### python train.py --img 640 --epochs 3 --data dataset.yaml --weights yolov5s.pt

In [ ]:
class Trainer:
    """
    Classe per addestrare un modello YOLOv5.
    """
    def __init__(self, model, dataset_yaml, img_size=640, batch_size=16, epochs=50, cache="ram"):
        """
        Inizializza il Trainer per YOLOv5.

        Args:
            model (YoloModel): Istanza del modello YOLOv5.
            dataset_yaml (str): Percorso al file di configurazione del dataset.
            img_size (int): Dimensione delle immagini di input.
            batch_size (int): Dimensione del batch per l'addestramento.
            epochs (int): Numero di epoche.
            cache (str): Tipo di caching ('ram' o 'disk').
        """
        self.model = model
        self.dataset_yaml = dataset_yaml
        self.img_size = img_size
        self.batch_size = batch_size
        self.epochs = epochs
        self.cache = cache

    def train(self): 
        """
        Avvia l'addestramento del modello utilizzando YOLOv5.
        """
        command = (
            f"python train.py --img {self.img_size} --batch-size {self.batch_size} "
            f"--epochs {self.epochs} --data {self.dataset_yaml} "
            f"--weights {self.model.model_path} --cache {self.cache}"
        )
        os.system(command)
        print("Addestramento completato.")

    def validate(self, weights_path=None):
        """
        Valida il modello sui dati di test.

        Args:
            weights_path (str, optional): Percorso ai pesi addestrati. Se None, usa i pesi attuali del modello.
        """
        weights = weights_path if weights_path else self.model.model_path
        command = f"python val.py --data {self.dataset_yaml} --weights {weights} --img {self.img_size}"
        os.system(command)
        print("Validazione completata.")
